### For filtering, spike detection and sorting to obtain single unit activity (discard MUAs)

In [ ]:
import pandas as pd
import os
import numpy as np
from scipy.signal import find_peaks, butter, filtfilt


#### Filtering